In [1]:
# Importation des modules utilisés
import sqlite3
import pandas as pd

# Création de la connexion
conn = sqlite3.connect(r"C:/Users/zara_/Downloads/ClassicModel.sqlite")

Ici, on effectue une jointure gauche (LEFT JOIN) entre la table Customers et Orders.
On sélectionne les clients (customerNumber, customerName) dont le numéro de commande (orderNumber) est NULL, 
ce qui signifie qu'ils n'ont pas de commande.

In [2]:
# Question 1: Lister les clients n’ayant jamais effectué de commande
clients_sans_commandes = pd.read_sql_query(
    """
    SELECT c.customerNumber, c.customerName
    FROM Customers c
    LEFT JOIN Orders o ON c.customerNumber = o.customerNumber
    WHERE o.customerNumber IS NULL
    """,
    conn)

display(clients_sans_commandes)  

,customerNumber,customerName
0,125,Havel & Zbyszek Co
1,168,American Souvenirs Inc
2,169,Porto Imports Co.
3,206,"Asian Shopping Network, Co"
4,223,Natürlich Autos
5,237,ANG Resellers
6,247,Messner Shopping Network
7,273,"Franken Gifts, Co"
8,293,BG&E Collectables
9,303,Schuyler Imports


Les statistiques par employé sont récupérées en faisant une jointure avec les tables Customers, Orders et OrderDetails. On utilise COUNT et SUM pour obtenir le nombre de clients, le nombre de commandes et le montant total des commandes pour chaque employé.

In [3]:
# Question 2: Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci
stats_employes = pd.read_sql_query(
 """
    SELECT e.employeeNumber, e.lastName, e.firstName,
        COUNT(DISTINCT c.customerNumber) AS nbclients,
        COUNT(o.orderNumber) AS nbcommandes,
        SUM(od.quantityOrdered * od.priceEach) AS totalcommandes
    FROM Employees e
    LEFT JOIN Customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN Orders o ON c.customerNumber = o.customerNumber
    LEFT JOIN OrderDetails od ON o.orderNumber = od.orderNumber
    GROUP BY e.employeeNumber
    ORDER BY e.employeeNumber;
    """,
    conn)

display(stats_employes)  

,employeeNumber,lastName,firstName,nbclients,nbcommandes,totalcommandes
0,1002,Murphy,Diane,0,0,NaN
1,1056,Patterson,Mary,0,0,NaN
2,1076,Firrelli,Jeff,0,0,NaN
3,1088,Patterson,William,0,0,NaN
4,1102,Bondur,Gerard,0,0,NaN
5,1143,Bow,Anthony,0,0,NaN
6,1165,Jennings,Leslie,6,331,1210228.57
7,1166,Thompson,Leslie,6,114,378064.72
8,1188,Firrelli,Julie,6,124,422257.44
9,1216,Patterson,Steve,6,152,565516.73


Question 3 : La jointure entre Offices, Employees, Customers, Orders et OrderDetails permet de calculer les statistiques pour chaque bureau. 
COALESCE est utilisé pour gérer les valeurs NULL dans les calculs de montant.

In [4]:
# Question 3: Pour chaque bureau, le nombre de clients, de commandes, montant total et clients d’un pays différent
stats_bureaux = pd.read_sql_query(
    """
    SELECT o.officeCode, o.city AS officeCity,COUNT(DISTINCT c.customerNumber) AS numberOfClients,
        COUNT(o2.orderNumber) AS numberOfOrders,
        COALESCE(SUM(od.quantityOrdered * od.priceEach), 0) AS totalSales,
        COUNT(DISTINCT CASE WHEN c.country != o.country THEN c.customerNumber END) AS clientsFromOtherCountries
    FROM Offices o
    LEFT JOIN Employees e ON o.officeCode = e.officeCode
    LEFT JOIN Customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN Orders o2 ON c.customerNumber = o2.customerNumber
    LEFT JOIN OrderDetails od ON o2.orderNumber = od.orderNumber
    GROUP BY o.officeCode, o.city
    ORDER BY o.officeCode;
    """,
    conn
)

display(stats_bureaux)  # Affichage des statistiques par bureau

,officeCode,officeCity,numberOfClients,numberOfOrders,totalSales,clientsFromOtherCountries
0,1.0,San Francisco,12,445,1588293.29,0
1,2.0,Boston,12,276,987774.17,0
2,3.0,NYC,15,353,1275993.95,3
3,4.0,Paris,29,959,3404055.56,17
4,5.0,Tokyo,5,137,503957.58,3
5,6.0,Sydney,10,370,1281705.83,5
6,7.0,London,17,456,1604168.80,12


Question 4:  La jointure entre Products, OrderDetails, Orders et Customers permet de calculer les statistiques pour chaque produit. On compte les commandes distinctes et la quantité totale commandée par produit.

In [5]:
# Question 4: Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents
produits = pd.read_sql_query(
    """
    SELECT p.productCode,p.productName, COUNT(DISTINCT o.orderNumber) AS nbcommandes,
        SUM(od.quantityOrdered) AS totalquantitecomm,
        COUNT(DISTINCT c.customerNumber) AS differents_clients
    FROM Products p
    LEFT JOIN OrderDetails od ON p.productCode = od.productCode
    LEFT JOIN Orders o ON od.orderNumber = o.orderNumber
    LEFT JOIN Customers c ON o.customerNumber = c.customerNumber
    GROUP BY p.productCode
    ORDER BY p.productCode;
    """,
    conn)

display(produits) 

,productCode,productName,nbcommandes,totalquantitecomm,differents_clients
0,S10_1678,1969 Harley Davidson Ultimate Chopper,28,1026.0,26
1,S10_1949,1952 Alpine Renault 1300,28,961.0,27
2,S10_2016,1996 Moto Guzzi 1100i,28,999.0,26
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,28,985.0,25
4,S10_4757,1972 Alfa Romeo GTA,28,1000.0,27
...,...,...,...,...,...
105,S700_3505,The Titanic,27,952.0,22
106,S700_3962,The Queen Mary,27,883.0,24
107,S700_4002,American Airlines: MD-11S,28,1073.0,26
108,S72_1253,Boeing X-32A JSF,28,960.0,27


Question 5 : On fait une jointure avec Orders, OrderDetails, et Payments pour récupérer les commandes et paiements par pays. 
Les clients sans commandes sont inclus grâce à l'utilisation de LEFT JOIN.

In [6]:
# Question 5: Donner le nombre de commande pour chaque pays, ainsi que le montant total des commandes et le montant total payé
display(pd.read_sql_query(
    """
    SELECT
        c.country AS Pays,
        COUNT(DISTINCT o.orderNumber) AS Nb_commande,
        IFNULL(SUM(od.priceEach * od.quantityOrdered), 0) AS Montant_tot_commandes,
        IFNULL(SUM(p.amount), 0) AS Total_paye
    FROM
        Customers c
    LEFT JOIN Orders o ON c.customerNumber = o.customerNumber
    LEFT JOIN OrderDetails od ON o.orderNumber = od.orderNumber
    LEFT JOIN Payments p ON c.customerNumber = p.customerNumber
    GROUP BY c.country
    ORDER BY c.country;
    """,
    conn)) 

,Pays,Nb_commande,Montant_tot_commandes,Total_paye
0,Australia,19,2182269.38,2.482541e+07
1,Austria,7,606187.59,4.090982e+06
2,Belgium,7,283705.44,1.931535e+06
3,Canada,7,448157.12,4.487022e+06
4,Denmark,7,781357.50,7.001114e+06
5,Finland,9,988745.73,1.009620e+07
6,France,37,3160296.75,3.141444e+07
7,Germany,7,576293.44,4.971661e+06
8,Hong Kong,2,48784.36,7.805498e+05
9,Ireland,2,115512.86,9.241029e+05


Question 6 : On crée une table de contingence en utilisant pivot_table() sur les résultats de la jointure entre 
Products, OrderDetails, Orders, et Customers, avec une agrégation sur le nombre de commandes par ligne de produit et pays.

In [7]:
# Question 6: Table de contingence du nombre de commandes entre la ligne de produits et le pays du client
display(pd.read_sql_query(
    """
    SELECT
        p.productLine AS ProductLine,
        c.country AS Pays,
        COUNT(DISTINCT o.orderNumber) AS nb_commandes
    FROM
        Customers c
    JOIN Orders o ON c.customerNumber = o.customerNumber
    JOIN OrderDetails od ON o.orderNumber = od.orderNumber
    JOIN Products p ON od.productCode = p.productCode
    GROUP BY p.productLine, c.country
    ORDER BY p.productLine, c.country;
    """,
    conn))

,ProductLine,Pays,nb_commandes
0,Classic Cars,Australia,12
1,Classic Cars,Austria,5
2,Classic Cars,Belgium,2
3,Classic Cars,Canada,6
4,Classic Cars,Denmark,5
...,...,...,...
121,Vintage Cars,Singapore,4
122,Vintage Cars,Spain,22
123,Vintage Cars,Sweden,4
124,Vintage Cars,UK,10


Question 7 : Semblable à la question 6, mais au lieu du nombre de commandes, 
on récupère le montant total payé par combinaison de ligne de produit et pays.

In [8]:
# Question 7: Table croisant la ligne de produits et le pays du client avec le montant total payé dans chaque cellule
display(pd.read_sql_query(
    """
    SELECT
        p.productLine AS ProductLine,
        c.country AS Country,
        IFNULL(SUM(pmt.amount), 0) AS TotalPaid
    FROM
        Customers c
    JOIN Orders o ON c.customerNumber = o.customerNumber
    JOIN OrderDetails od ON o.orderNumber = od.orderNumber
    JOIN Products p ON od.productCode = p.productCode
    LEFT JOIN Payments pmt ON c.customerNumber = pmt.customerNumber
    GROUP BY p.productLine, c.country
    ORDER BY p.productLine, c.country;
    """,
    conn
))

,ProductLine,Country,TotalPaid
0,Classic Cars,Australia,7504795.97
1,Classic Cars,Austria,1884419.42
2,Classic Cars,Belgium,166880.87
3,Classic Cars,Canada,774924.01
4,Classic Cars,Denmark,3678313.22
...,...,...,...
121,Vintage Cars,Singapore,1941227.28
122,Vintage Cars,Spain,39878490.66
123,Vintage Cars,Sweden,1377094.16
124,Vintage Cars,UK,4673162.78


Question 8 : On calcule la marge moyenne pour chaque produit en soustrayant le buyPrice du priceEach dans OrderDetails. Les produits sont triés par cette marge décroissante.

In [9]:
# Question 8: Produits ayant la marge moyenne la plus élevée
display(pd.read_sql_query(
    """
    SELECT p.productCode, p.productName, AVG(od.priceEach - p.buyPrice) AS marge_moyenne
    FROM Products p
    JOIN OrderDetails od ON p.productCode = od.productCode
    GROUP BY p.productCode, p.productName
    ORDER BY marge_moyenne DESC
    LIMIT 10;
    """,
    conn
))  # Affichage des produits avec la marge moyenne la plus élevée

,productCode,productName,marge_moyenne
0,S10_1949,1952 Alpine Renault 1300,99.006429
1,S10_4698,2003 Harley-Davidson Eagle Drag Bike,95.235000
2,S18_3232,1992 Ferrari 360 Spider red,83.334906
3,S12_2823,2002 Suzuki XREO,83.201429
4,S18_2795,1928 Mercedes-Benz SSK,82.696786
5,S12_1108,2001 Ferrari Enzo,81.043704
6,S12_3891,1969 Ford Falcon,77.335926
7,S18_3685,1948 Porsche Type 356 Roadster,72.636800
8,S18_2870,1999 Indy 500 Monte Carlo SS,71.794400
9,S18_1749,1917 Grand Touring Sedan,70.432800


Question 9 : Les produits vendus à perte sont récupérés en filtrant les produits dont le priceEach est inférieur au buyPrice. 
Pour chaque vente à perte, les informations sont affichées.

In [10]:
# Question 9: Lister les produits vendus à perte
display(pd.read_sql_query(
    """
    SELECT p.productCode AS ProductCode,p.productName AS ProductName,c.customerName AS CustomerName,o.customerNumber AS CustomerNumber,
        od.priceEach AS SalePrice,
        p.buyPrice AS PurchasePrice
    FROM
        OrderDetails od
    JOIN Products p ON od.productCode = p.productCode
    JOIN Orders o ON od.orderNumber = o.orderNumber
    JOIN Customers c ON o.customerNumber = c.customerNumber
    WHERE
        od.priceEach < p.buyPrice
    ORDER BY
        c.customerName, p.productName;
    """,
    conn
))  

,ProductCode,ProductName,CustomerName,CustomerNumber,SalePrice,PurchasePrice
0,S18_3136,18th Century Vintage Horse Carriage,"AV Stores, Co.",187,39.80,60.74
1,S18_2795,1928 Mercedes-Benz SSK,"AV Stores, Co.",187,52.67,72.56
2,S24_2887,1952 Citroen-15CV,"AV Stores, Co.",187,42.26,72.82
3,S24_3191,1969 Chevrolet Camaro Z28,"AV Stores, Co.",187,34.19,50.51
4,S24_3432,2002 Chevy Corvette,"AV Stores, Co.",187,37.18,62.11
...,...,...,...,...,...,...
74,S10_2016,1996 Moto Guzzi 1100i,"Tokyo Collectables, Ltd",398,68.92,68.99
75,S10_4698,2003 Harley-Davidson Eagle Drag Bike,"Tokyo Collectables, Ltd",398,76.67,91.02
76,S18_1589,1965 Aston Martin DB5,Toys4GrownUps.com,205,56.30,65.96
77,S18_2319,1964 Mercedec Tour Bus,Vitachrome Inc.,181,37.48,74.86


Quesiton 10 On compare le montant total payé par chaque client avec le coût total des produits achetés. 
Les clients ayant payé plus que le coût sont affichés.

In [11]:
# Question 10: Lister les clients pour lesquels le montant total payé est supérieur aux montants totaux des achats
display(pd.read_sql_query(
    """
    SELECT c.customerNumber,c.customerName,
        COALESCE(SUM(od.priceEach * od.quantityOrdered), 0) AS totalPaid,
        COALESCE(SUM(p.buyPrice * od.quantityOrdered), 0) AS totalCost
    FROM Customers c
    JOIN Orders o ON c.customerNumber = o.customerNumber
    JOIN OrderDetails od ON o.orderNumber = od.orderNumber
    JOIN Products p ON od.productCode = p.productCode
    GROUP BY c.customerNumber, c.customerName
    HAVING totalPaid > totalCost
    ORDER BY totalPaid DESC;
    """,
    conn))  

,customerNumber,customerName,totalPaid,totalCost
0,141,Euro+ Shopping Channel,912294.11,494169.88
1,124,Mini Gifts Distributors Ltd.,654858.04,355057.95
2,114,"Australian Collectors, Co.",200995.41,110274.00
3,151,Muscle Machine Inc,197736.94,105543.86
4,119,La Rochelle Gifts,180124.94,97697.82
...,...,...,...,...
93,456,Microscale Inc.,33144.93,16164.41
94,473,Frau da Collezione,28951.91,15825.39
95,198,Auto-Moto Classics Inc.,26479.26,14968.24
96,103,Atelier graphique,24179.96,12250.56


In [12]:
# Fermeture de la connexion
conn.close()